In [ ]:
#!g1.1
%pip install -r requirements.txt

In [2]:
#!g1.1
%pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [3]:
#!g1.1
%pip install torchtext

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 881.9 MB 2.0 kB/s              
  Attempting uninstall: torch
    Found existing installation: torch 1.9.1
    Uninstalling torch-1.9.1:
      Successfully uninstalled torch-1.9.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.1+cu111 requires torch==1.9.1, but you have torch 1.10.1 which is incompatible.
torchaudio 0.9.1 requires torch==1.9.1, but you have torch 1.10.1 which is incompatible.


In [4]:
#!g1.1
%pip install razdel

Defaulting to user installation because normal site-packages is not writeable


## Цель тетрадки обучить модель и сохранить ее для бота

* сначала надо обернуть данные мейл-ру в datasets

Сначала проредим данные, чтобы не тратить кучу времени на обучение

In [19]:
#!g1.1
!unzip train.txt.zip

Archive:  train.txt.zip
  inflating: train.txt               


In [21]:
#!g1.1
!cat train.txt | awk 'BEGIN{i=0} {i+=1; if (i % 50 == 0) {print($0)} ;  }' >> rared_train.txt

In [5]:
#!g1.1
import os
import numpy as np
import pandas as pd
import json
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from typing import Optional
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
#!g1.1
class PairsDataset(Dataset):
    def __init__(self, file_path, sep='\t'):
        super().__init__()
        
        self.file_path = file_path
        self.sep = sep
        self.data = self.load_data()
        
    def load_data(self): 
        data = list()
        with open(self.file_path) as file_object:
            for line in file_object:
                data.append(line.strip().split(self.sep))
        return data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

In [94]:
#!g1.1
data = PairsDataset('rared_train.txt')

In [8]:
#!g1.1
round(0.2 * data.__len__())

23522

In [9]:
#!g1.1
split = 0.2
test_data = data[:round(split * data.__len__())]
train_data = data[round(split * data.__len__()):]

In [10]:
#!g1.1
train_data.__len__()

94087

Еще немного предобработки данных:

In [95]:
#!g1.1
class Collater:
    def __init__(self, 
                 tokenizer_name='sberbank-ai/rugpt3small_based_on_gpt2', 
                 max_length=24):
        
        # будем использовать предобученный токенизатор
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        # устанавливаем пад токен, он будет иметь индекс "длина словаря + 1"
        self.tokenizer.pad_token = '<pad>'
        self.tokenizer.bos_token = '<bos>'
        self.tokenizer.eos_token = '<eos>'
        self.max_length = max_length
        
    def tokenize(self, texts):
        tokenized = self.tokenizer(texts, padding=True, 
                                   truncation=True,
                                   max_length=self.max_length, 
                                   return_tensors='pt')
        # за пару минут я не нашел как вставить пад на нужное мне место, поэтому прибегнул к такому хаку
        tokenized['input_ids'] = tokenized['input_ids'] * tokenized['attention_mask']
        return tokenized
    
    def __call__(self, batch):
        
        questions, responses = list(), list()
        
        for el in batch:
            #el[0]: str
            questions.append('<bos> ' + el[0] + ' <eos>')
            responses.append('<bos> ' + el[1] + ' <eos>')
            
        tokenized_questions = self.tokenize(questions)
        tokenized_responses = self.tokenize(responses)
        
        return (tokenized_questions, tokenized_responses)

In [56]:
#!g1.1
512/2

256.0

In [96]:
#!g1.1
collater = Collater(tokenizer_name='sberbank-ai/rugpt3small_based_on_gpt2')
train_loader = DataLoader(train_data, batch_size=256, shuffle=True, collate_fn=collater)

In [61]:
#!g1.1
collater.tokenize('<bos> Котик <eos>')

{'input_ids': tensor([[  32,   70, 1410,   34, 2839, 4415, 8483,   73, 1410,   34]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [97]:
#!g1.1
train_out = next(iter(train_loader))
print(train_out[1].keys())
print(f"Question batch shape: {train_out[0]['input_ids'].size()}")
print(f"Answer batch shape: {train_out[1]['input_ids'].size()}")

dict_keys(['input_ids', 'attention_mask'])
Question batch shape: torch.Size([256, 24])
Answer batch shape: torch.Size([256, 24])


In [63]:
#!g1.1
from torchtext.vocab import build_vocab_from_iterator
from razdel import tokenize

def yield_tokens(data_iter):
    for pair in data_iter:
        src_tokens, trg_tokens = list(tokenize(pair[0])), list(tokenize(pair[1]))
        src_text_tokens, trg_text_tokens = [_.text for _ in src_tokens], [_.text for _ in trg_tokens]
        src_text_tokens.extend(trg_text_tokens)
        yield src_text_tokens
        
vocab = build_vocab_from_iterator(yield_tokens(train_data), specials=['<pad>', '<unk>', '<bos>', '<eos>'])
vocab.set_default_index(vocab["<unk>"])

In [78]:
#!g1.1
# test
vocab(["<bos>", "работает", "ли", "эта", "штукенция", "<eos>", "<unk>", "<pad>"])

[2, 781, 25, 346, 1, 3, 1, 0]

In [103]:
#!g1.1
#test 
vocab.get_stoi()["работает"]

781

In [106]:
#!g1.1
vocab.get_itos()[700]

'пор'

Disclaimer:

пользовалась вот этим туториалом https://www.youtube.com/watch?v=U0s0f995w14&ab_channel=AladdinPersson

In [65]:
#!g1.1
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        # Get number of training examples
        N = query.shape[0]

        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        query = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)  # (N, value_len, heads, head_dim)
        keys = self.keys(keys)  # (N, key_len, heads, head_dim)
        queries = self.queries(query)  # (N, query_len, heads, heads_dim)

        # Einsum does matrix mult. for query*keys for each training example
        # with every other training example, don't be confused by einsum
        # it's just how I like doing matrix multiplication & bmm

        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys])
        # queries shape: (N, query_len, heads, heads_dim),
        # keys shape: (N, key_len, heads, heads_dim)
        # energy: (N, heads, query_len, key_len)

        # Mask padded indices so their weights become 0
        if mask is not None:
            energy = energy.masked_fill(mask == 0, float("-1e20"))

        # Normalize energy values similarly to seq2seq + attention
        # so that they sum to 1. Also divide by scaling factor for
        # better stability
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        # attention shape: (N, heads, query_len, key_len)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        # attention shape: (N, heads, query_len, key_len)
        # values shape: (N, value_len, heads, heads_dim)
        # out after matrix multiply: (N, query_len, heads, head_dim), then
        # we reshape and flatten the last two dimensions.

        out = self.fc_out(out)
        # Linear layer doesn't modify the shape, final shape will be
        # (N, query_len, embed_size)

        return out

In [66]:
#!g1.1
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out


In [67]:
#!g1.1
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embed_size,
                 num_layers, heads, device,
                 forward_expansion, dropout, max_length):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )

        # In the Encoder the query, key, value are all the same, it's in the
        # decoder this will change. This might look a bit odd in this case.
        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out

In [68]:
#!g1.1
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads=heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out


class Decoder(nn.Module):
    def __init__(self, trg_vocab_size,
                 embed_size, num_layers, heads,
                 forward_expansion, dropout, device, max_length):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)

        return out

In [69]:
#!g1.1
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx,
                 embed_size=512, num_layers=6, forward_expansion=4,
                 heads=8, dropout=0, device="cpu", max_length=100):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        # (N, 1, 1, src_len)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )

        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

In [41]:
#!g1.1

# test

x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(device)
trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

src_pad_idx = 0
trg_pad_idx = 0
src_vocab_size = 10
trg_vocab_size = 10
model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(device)
out = model(x, trg[:, :-1])
print(out.shape)

AttributeError: 'tuple' object has no attribute 'shape'

* обучение модели

In [98]:
#!g1.1
from torch.utils.tensorboard import SummaryWriter


vocab_size = len(vocab)
src_vocab_size = vocab_size
trg_vocab_size = vocab_size
src_pad_idx = vocab["<pad>"]
trg_pad_idx = vocab["<pad>"]

model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx,
                    embed_size=512, num_layers=3, forward_expansion=4,
                    heads=8, dropout=0, device=device, max_length=100).to(device)

num_epochs = 5
learning_rate = 3e-4

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.1, patience=10, verbose=True
)
    
criterion = nn.CrossEntropyLoss(ignore_index=src_pad_idx)

In [279]:
#!g1.1
for n_iter, (question, response) in enumerate(train_loader):
    print(question['attention_mask'])

In [111]:
#!g1.1
tokenize(sentence)

<generator object find_substrings at 0x7f54ec0c8b30>

In [119]:
#!g1.1
from razdel import tokenize

stoi = vocab.get_stoi()
itos = vocab.get_itos()

def translate_sentence(model, sentence, device, max_length=50):
    # Create tokens nd everything in lower case (which is what our vocab is)
    
    print(sentence)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in list(tokenize(sentence))]
    else:
        tokens = [token.lower() for token in sentence]

    # Add <BOS> and <EOS> in beginning and end respectively
    tokens.insert(0, '<bos>')
    tokens.append('<eos>')
    print(tokens)

    # Go through each src token and convert to an index
    #text_to_indices = [stoi[token] for token in tokens]
    
    sentence_tensor = collater.tokenize(sentence)

    # Convert to Tensor
    #sentence_tensor = torch.LongTensor(text_to_indices['input_ids']).unsqueeze(1).to(device)

    outputs = [stoi['<bos>']]
    
    trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

    with torch.no_grad():
        output = model(sentence_tensor['input_ids'], trg_tensor)

    best_guess = output.argmax(2)[-1, :].item()
    outputs.append(best_guess)

    #    if best_guess == stoi['<eos>']:
    #        break

    translated_sentence = [itos[idx] for idx in outputs]
    # remove start token
    return translated_sentence[1:]


In [101]:
#!g1.1
from tqdm import tqdm


collater = Collater(tokenizer_name='sberbank-ai/rugpt3small_based_on_gpt2')

model.train()

for n_epoch in range(1, num_epochs + 1):
    
    progress_bar = tqdm(total=len(train_loader), desc=f'Epoch {n_epoch}')
    
    sentence = 'В каком университете ты учишься?'
    
    #model.eval()
    #translated_sentence = translate_sentence(
    #    model, sentence, device, max_length=50
    #)

    #print(f"Translated example sentence: \n {translated_sentence}")
    #model.train()
    
    losses = list()
    
    for n_iter, (question, response) in enumerate(train_loader):
        
        #question = batch_to_device(question, device)
        #response = batch_to_device(response, device)
        question = question['input_ids'].to(device)
        response = response['input_ids'].to(device)
        
        #question_embeddings, response_embeddings = model(question, response)
        response_embeddings = model(question, response)
        #print(out.size())   # 512, 24, 117016 batch_size, max_size in collater, vocab_size
        response_embeddings = response_embeddings.to(device)
        #print(response_embeddings.size()) 
        
        #idea
        #outputs_flatten = outputs[1:].view(-1, outputs.shape[-1])
        #trg_flatten = batch.trg[1:].view(-1)
        
        #bad idea
        #question_embeddings = question_embeddings[1:].reshape(-1, question_embeddings.shape[2])
        #response_embeddings = response_embeddings.reshape(response_embeddings.shape[0], -1)
        
        output = response_embeddings[1:].reshape(-1, response_embeddings.shape[2])
        target = response[1:].reshape(-1)
        
        optimizer.zero_grad()
        
        loss = criterion(output.to(device), target.to(device))
        
        losses.append(loss.item())
        
        # Back prop
        loss.backward()
        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        
        # Gradient descent step
        optimizer.step()
        
        progress_bar.update()
        progress_bar.set_postfix(loss=np.mean(losses[-100:]))
    
    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)
    torch.save(model.state_dict(), f'transformer_state_dict_{n_epoch}.pth')

    progress_bar.close()

Epoch 1:   0%|          | 0/368 [02:58<?, ?it/s]
/kernel/lib/python3.8/site-packages/ml_kernel/ignored_keyboard_interrupt.py:16: UserWarning: Kernel cannot be interrupted during state load
  warnings.warn(self._warn_message)
/kernel/lib/python3.8/site-packages/ml_kernel/ignored_keyboard_interrupt.py:16: UserWarning: Kernel cannot be interrupted during state load
  warnings.warn(self._warn_message)
/kernel/lib/python3.8/site-packages/ml_kernel/ignored_keyboard_interrupt.py:16: UserWarning: Kernel cannot be interrupted during state load
  warnings.warn(self._warn_message)
Epoch 4: 100%|██████████| 368/368 [02:18<00:00,  3.05it/s, loss=0.0184]

OSError: [Errno 28] No space left on device

/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:862: UserWarning: The following variables cannot be serialized: progress_bar
  warnings.warn(message)


```
Epoch 1: 100%|██████████| 368/368 [05:05<00:00,  1.21it/s, loss=0.442]]
Epoch 2: 100%|██████████| 368/368 [02:27<00:00,  2.49it/s, loss=0.119]
Epoch 3: 100%|██████████| 368/368 [02:28<00:00,  2.48it/s, loss=0.044] 
Epoch 4: 100%|██████████| 368/368 [02:18<00:00,  3.05it/s, loss=0.0184]
```

## Итого:
    
У меня получилось сделать один луп обучение модели на сокращенных данных, где я брала только каждое 50-ое вхождение из исходных данных. Еще размер батча вынужденно 256, потому что 512 не проходило по ограничением памяти. Делалось это всё в яндекс датасфере.

Дальше надо было реализовать нормальное обучение модели, то есть взять побольше данных и оставить ее часа на два, но это сделать я, к сожалению, уже не успела.

Метод генерации ответа, я написала, но не до конца его дотестировала.

In [120]:
#!g1.1
model.eval()

sentence = 'В каком университете ты учишься?'
translated_sentence = translate_sentence(
    model, sentence, device, max_length=50
)

print(f"Translated example sentence: \n {translated_sentence}")

В каком университете ты учишься?
['<bos>', 'в', 'каком', 'университете', 'ты', 'учишься', '?', '<eos>']


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

/kernel/lib/python3.8/site-packages/ml_kernel/kernel.py:862: UserWarning: The following variables cannot be serialized: model, optimizer, output, scheduler
  warnings.warn(message)
